In [97]:
from src.nws import NWSClient, celsius_to_fahrenheit, fahrenheit_to_celsius
from src.params import *
from datetime import datetime, timedelta
import pytz

client = NWSClient()

In [104]:
station = StationID.NYC
radii = {
    StationID.MDW: 8,
    StationID.MIA: 5,
    StationID.NYC: 4,
    StationID.AUS: 5
}
station_tz = STATION_TZ[station]
prior_days = 0.5

dfs = client.get_timeseries(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ), radius=radii[station])

for df in dfs:
    df.rename(columns={
        'air_temp_set_1': 'temperature', 
        'relative_humidity_set_1': 'relative_humidity',
        'wind_speed_set_1': 'wind_speed',
        'wind_direction_set_1': 'wind_direction',
        'air_temp_high_6_hour_set_1': 'temperature_high_6_hour',
        'air_temp_high_24_hour_set_1': 'temperature_high_24_hour',
    }, inplace=True)

    df['temperature_f'] = celsius_to_fahrenheit(df['temperature'])

[df.info() for df in dfs]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12 entries, 2024-11-15 04:51:00-05:00 to 2024-11-15 15:51:00-05:00
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   temperature              12 non-null     float64
 1   relative_humidity        12 non-null     float64
 2   wind_speed               10 non-null     float64
 3   wind_direction           6 non-null      float64
 4   temperature_high_6_hour  2 non-null      float64
 5   time                     12 non-null     object 
 6   station_id               12 non-null     object 
 7   station_name             12 non-null     object 
 8   distance                 12 non-null     float64
 9   temperature_f            12 non-null     float64
dtypes: float64(7), object(3)
memory usage: 1.0+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2024-11-15 07:00:00-05:00 to 2024-11-15 07:00:00-05:00
Data columns (total 7 column

[None, None, None, None]

In [105]:
import pandas as pd
from functools import reduce

res_df = pd.concat([df.reset_index() for df in dfs], axis=0, ignore_index=True)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype                           
---  ------                    --------------  -----                           
 0   date_time                 201 non-null    datetime64[ns, America/New_York]
 1   temperature               201 non-null    float64                         
 2   relative_humidity         200 non-null    float64                         
 3   wind_speed                198 non-null    float64                         
 4   wind_direction            187 non-null    float64                         
 5   temperature_high_6_hour   2 non-null      float64                         
 6   time                      201 non-null    object                          
 7   station_id                201 non-null    object                          
 8   station_name              201 non-null    object                          
 9   distance  

In [106]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

fig = px.line(res_df, x="date_time", y="temperature_f", color="station_name", title=f"Temperature (°F) for Station {station}", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()